## «Крестики-нолики»: смена игроков после каждого хода

Чтобы реализовать логику смены игрока, нужно доработать код в файле game.py следующим образом:

 1. Добавить переменную, в которой по умолчанию будет храниться значение `X`, так как в игре «Крестики-нолики» первый ход обычно делают крестиками. Подходящее название для этой переменной — `current_player`.
 2. Добавить `флаговую переменную` `running` со значением по умолчанию `True`. Она понадобится для управления основным циклом игры. Пока поднят флаг (`running = True`) — игра продолжается. Если флаг опущен (running = False) — цикл останавливается, и игра заканчивается.
 3. Реализовать смену игроков. Эта возможность должна работать следующим образом: в каждой новой итерации цикла, если `current_player` равен `X`, новым значением будет `O`, иначе — новым значением будет `X`. Для реализации такого описания подойдёт тернарный оператор: `current_player = 'O' if current_player == 'X' else 'X'`.

Изучите листинг и комментарии к нему, а затем доработайте код в файле *game.py*: 

In [ ]:
# game.py

from gameparts import Board
from gameparts.exceptions import FieldIndexError


def main():
    game = Board()
    # Первыми ходят крестики.
    current_player = 'X'
    # Это флаговая переменная. По умолчанию игра запущена и продолжается.
    running = True
    game.display()

    # Тут запускается основной цикл игры.
    while running:

        print(f'Ход делают {current_player}')

        # Запускается бесконечный цикл.
        while True:
            try:
                row = int(input('Введите номер строки: '))
                if row < 0 or row >= game.field_size:
                    raise FieldIndexError
                column = int(input('Введите номер столбца: '))
                if column < 0 or column >= game.field_size:
                    raise FieldIndexError
            except FieldIndexError:
                print(
                    'Значение должно быть неотрицательным и меньше '
                    f'{game.field_size}.'
                )
                print('Введите значения для строки и столбца заново.')
                continue
            except ValueError:
                print('Буквы вводить нельзя. Только числа.')
                print('Введите значения для строки и столбца заново.')
                continue
            except Exception as e:
                print(f'Возникла ошибка: {e}')
            else:
                break

        # Теперь для установки значения на поле само значение берётся
        # из переменной current_player.
        game.make_move(row, column, current_player)
        game.display()
        # Тернарный оператор, через который реализована смена игроков.
        # Если current_player равен X, то новым значением будет O,
        # иначе — новым значением будет X.
        current_player = 'O' if current_player == 'X' else 'X'


if __name__ == '__main__':
    main()

***
## Ещё одно собственное исключение

Создайте собственное исключение `CellOccupiedError`. Обновите код в файле *exceptions.py*:

In [ ]:
# gameparts/exceptions.py

class FieldIndexError(IndexError):
    """ Выбрасывается, если выбрано значение вне поля. """

    def __init__(
            self,
            message='Введено значение за границами игрового поля!'
        ):
        
        super().__init__(message)

# Вот оно - новое исключение, унаследованное от базового класса Exception.
class CellOccupiedError(Exception):
    def __init__(
            self,
            message='Попытка изменить занятую ячейку'
        ):
        
        super().__init__(message)


Теперь вы можете вызвать это исключение в основном коде игры. Обновите код в файле *game.py*:

In [ ]:
# game.py

from gameparts import Board
# Добавился ещё один импорт - исключение CellOccupiedError.
from gameparts.exceptions import CellOccupiedError, FieldIndexError

def main():
    game = Board()
    current_player = 'X'
    running = True
    game.display()

    while running:

        print(f'Ход делают {current_player}')

        # Запускается бесконечный цикл.
        while True:
            try:
                row = int(input('Введите номер строки: '))
                if row < 0 or row >= game.field_size:
                    raise FieldIndexError
                column = int(input('Введите номер столбца: '))
                if column < 0 or column >= game.field_size:
                    raise FieldIndexError
                if game.board[row][column] != ' ':
                    # Вот тут выбрасывается новое исключение.
                    raise CellOccupiedError
            except FieldIndexError:
                print(
                    'Значение должно быть неотрицательным и меньше '
                    f'{game.field_size}.'
                )
                print('Введите значения для строки и столбца заново.')
                continue
            except CellOccupiedError:
                print('Ячейка занята')
                print('Введите другие координаты.')
                continue
            except ValueError:
                print('Буквы вводить нельзя. Только числа.')
                print('Введите значения для строки и столбца заново.')
                continue
            except Exception as e:
                print(f'Возникла ошибка: {e}')
            else:
                break

        game.make_move(row, column, current_player)
        game.display()
        current_player = 'O' if current_player == 'X' else 'X'

if __name__ == '__main__':
    main()

***
## «Крестики-нолики»: ничья или победа

In [ ]:
# gameparts/parts.py

...

    def display(self):
        ...

    def is_board_full(self):
        # Цикл проходится по всем столбцам игрового поля.
        for i in range(self.field_size):
            # А потом по всем строчкам.
            for j in range(self.field_size):
                # Если находит свободную ячейку...
                if self.board[i][j] == ' ':
                    # ...игра продолжается.
                    return False
        # Иначе - ничья!
        return True

...

Чтобы программа могла определить победу, нужно:

1. Реализовать проверку по вертикали и горизонтали с помощью цикла. Он будет проходиться по каждой строке и столбцу и проверять, одинаковые ли в них символы.
2. Реализовать проверку по диагонали с помощью условия. Нужно сравнить символы в определённых позициях поля. Например, для главной диагонали — это `[0][0]`, `[1][1]`, `[2][2]`, а для побочной — `[0][2]`, `[1][1]`, `[2][0]`.
3. В основном цикле игры после каждого хода нужно запускать обе проверки и если хотя бы одна из них закончится успешно, то опускать флаг `running` и выводить результат игры в терминал.

In [ ]:
# gameparts/parts.py

...

    # Этот метод будет определять победу.
    def check_win(self, player):
        # Проверка по горизонталям и вертикалям
        for i in range(self.field_size):
            if (all([self.board[i][j] == player for j in range(self.field_size)]) or
                    all([self.board[j][i] == player for j in range(self.field_size)])):
                return True
    
        # Проверка по диагоналям
        if (all([self.board[i][i] == player for i in range(self.field_size)]) or
                all([self.board[i][self.field_size - 1 - i] == player for i in range(self.field_size)])):
            return True
    
        return False


...

In [ ]:
# game.py

from gameparts import Board
from gameparts.exceptions import CellOccupiedError, FieldIndexError

def main():
    game = Board()
    current_player = 'X'
    running = True
    game.display()

    while running:

        print(f'Ход делают {current_player}')

        while True:
            try:
                row = int(input('Введите номер строки: '))
                if row < 0 or row >= game.field_size:
                    raise FieldIndexError
                column = int(input('Введите номер столбца: '))
                if column < 0 or column >= game.field_size:
                    raise FieldIndexError
                if game.board[row][column] != ' ':
                    raise CellOccupiedError
            except FieldIndexError:
                print(
                    'Значение должно быть неотрицательным и меньше '
                    f'{game.field_size}.'
                )
                print('Введите значения для строки и столбца заново.')
                continue
            except CellOccupiedError:
                print('Ячейка занята.')
                print('Пожалуйста, введите другие координаты.')
                continue
            except ValueError:
                print('Буквы вводить нельзя. Только числа.')
                print('Введите значения для строки и столбца заново.')
                continue
            except Exception as e:
                print(f'Возникла ошибка: {e}')
            else:
                break

        game.make_move(row, column, current_player)
        game.display()
        # После каждого хода надо делать проверку на победу и на ничью.
        if game.check_win(current_player):
            print(f'Победили {current_player}.')
            running = False
        elif game.is_board_full():
            print('Ничья!')
            running = False

        current_player = 'O' if current_player == 'X' else 'X'

if __name__ == '__main__':
    main()